# Which Tech Giant Is Best Positioned for a Downturn?

## A Financial Health Analysis of Six Major Technology Companies

---

### The Question

When the economy slows down, not all companies are equally prepared. Some have cash stockpiles and low debt. Others are leveraged and dependent on continued growth to stay afloat.

This analysis examines six major tech companies to answer: **If a recession hit tomorrow, who would be in the strongest position to survive—and even thrive?**

### The Companies

| Company | Ticker | Why Include Them |
|---------|--------|------------------|
| Apple | AAPL | Consumer hardware giant, massive cash reserves |
| Microsoft | MSFT | Enterprise software, recurring revenue model |
| Alphabet | GOOGL | Advertising dependent, but diversifying |
| Meta | META | Advertising dependent, recent cost restructuring |
| Nvidia | NVDA | Semiconductor leader, riding the AI wave |
| Intel | INTC | Semiconductor incumbent, facing challenges |

### How We'll Measure "Downturn Readiness"

We'll focus on three categories of financial metrics:

**1. Liquidity** — Can they pay their bills if revenue drops?
- Current Ratio
- Quick Ratio

**2. Leverage** — How much debt are they carrying?
- Debt-to-Equity Ratio
- Interest Coverage Ratio

**3. Cash Flow** — Do they generate real cash, not just accounting profits?
- Free Cash Flow
- FCF Margin
- Cash on Hand

### Data Source

All financial data is pulled from Yahoo Finance using the `yfinance` Python library. This gives us access to income statements, balance sheets, and cash flow statements for public companies.

---

## Part 1: Setup and Data Collection

Before we analyze anything, we need to:
1. Import the libraries we'll use
2. Confirm that yfinance is working properly
3. Understand what data we're working with

### 1.1 Import Libraries

We'll use four main libraries:
- **yfinance** — pulls financial data from Yahoo Finance
- **pandas** — for organizing and manipulating data in tables
- **matplotlib** — for creating charts
- **seaborn** — makes matplotlib charts look better with less effort

In [ ]:
# ============================================================
# INSTALL REQUIRED PACKAGES
# ============================================================
# The exclamation mark tells Jupyter to run this as a terminal command
# These will install the packages if you don't have them already

!pip install yfinance --quiet
!pip install pandas --quiet
!pip install matplotlib --quiet
!pip install seaborn --quiet

# ============================================================
# IMPORT LIBRARIES
# ============================================================

# yfinance lets us pull financial data from Yahoo Finance
# We'll use it to get income statements, balance sheets, and cash flow statements
import yfinance as yf

# pandas is the go-to library for working with tabular data
# It gives us DataFrames, which are like Excel spreadsheets in Python
import pandas as pd

# matplotlib is the core plotting library
# pyplot is the module we use most often for creating charts
import matplotlib.pyplot as plt

# seaborn builds on matplotlib and makes statistical charts easier
# It also has nicer default styling
import seaborn as sns

# warnings can clutter our output, so we'll suppress non-critical ones
import warnings
warnings.filterwarnings('ignore')

# Set some display options so our tables look clean
pd.set_option('display.float_format', '{:.2f}'.format)  # 2 decimal places
pd.set_option('display.max_columns', 15)                # Show more columns

# Use seaborn's clean style for all our charts
sns.set_style('whitegrid')

print("All libraries imported successfully.")

### 1.2 Test: Can We Pull Data?

Before we try to pull data for all six companies, let's make sure yfinance is working. We'll pull data for just Apple (AAPL) and see what we get back.

This also helps us understand the structure of the data before we write more complex code.

In [ ]:
# ============================================================
# TEST: PULL DATA FOR ONE COMPANY
# ============================================================

# Create a Ticker object for Apple
# This is like opening a connection to Apple's financial data
test_ticker = yf.Ticker('AAPL')

# Let's see what basic info we can get
# The .info property returns a dictionary with company details
test_info = test_ticker.info

# Print a few key pieces to confirm it's working
print("Company Name:", test_info.get('longName', 'Not found'))
print("Sector:", test_info.get('sector', 'Not found'))
print("Market Cap:", f"${test_info.get('marketCap', 0) / 1e9:.1f} billion")
print("\nyfinance is working!")

---

## Part 2: Exploring the Data Structure

Now that we know yfinance works, we need to understand what data we're getting. Financial statements from yfinance come as pandas DataFrames, but the field names aren't always intuitive.

Let's look at each statement type to find the exact field names we need for our ratios.

### 2.1 Income Statement

The income statement shows revenue, costs, and profits over a period (usually a year or quarter). We'll need this for:
- **Operating Income** — used in Interest Coverage ratio
- **Total Revenue** — used in FCF Margin

In [ ]:
# ============================================================
# EXPLORE: INCOME STATEMENT
# ============================================================

# Pull the annual income statement for Apple
# .financials gives us the income statement (confusing name, I know)
income_statement = test_ticker.financials

# Let's see what fields are available
# The index contains all the line item names (Revenue, Net Income, etc.)
print("Income Statement Fields:")
print("=" * 40)
for field in income_statement.index:
    print(f"  - {field}")

In [ ]:
# Let's also see the actual data structure
# Columns are dates (fiscal year ends), rows are line items
print("Income Statement Shape:", income_statement.shape)
print(f"\nWe have {income_statement.shape[0]} line items across {income_statement.shape[1]} years")
print("\nYears available (most recent first):")
for col in income_statement.columns:
    print(f"  - {col.strftime('%Y-%m-%d')}")

### 2.2 Balance Sheet

The balance sheet shows what a company owns (assets) and owes (liabilities) at a specific point in time. We'll need this for:
- **Current Assets** and **Current Liabilities** — for Current Ratio and Quick Ratio
- **Total Debt** and **Stockholders Equity** — for Debt-to-Equity
- **Cash and Cash Equivalents** — for cash on hand

In [ ]:
# ============================================================
# EXPLORE: BALANCE SHEET
# ============================================================

# Pull the annual balance sheet
balance_sheet = test_ticker.balance_sheet

# See what fields are available
print("Balance Sheet Fields:")
print("=" * 40)
for field in balance_sheet.index:
    print(f"  - {field}")

### 2.3 Cash Flow Statement

The cash flow statement shows actual cash moving in and out of the business. This is crucial because profits on paper don't always mean cash in the bank. We'll need this for:
- **Operating Cash Flow** — cash generated from the core business
- **Capital Expenditure** — cash spent on equipment, buildings, etc.
- **Free Cash Flow** = Operating Cash Flow - Capital Expenditure

In [ ]:
# ============================================================
# EXPLORE: CASH FLOW STATEMENT
# ============================================================

# Pull the annual cash flow statement
cash_flow = test_ticker.cashflow

# See what fields are available
print("Cash Flow Statement Fields:")
print("=" * 40)
for field in cash_flow.index:
    print(f"  - {field}")

### 2.4 Summary: Fields We Need

Based on our exploration above, let's map out exactly which fields we need for each ratio. Run this cell after you've seen the field names above, and we'll confirm they exist.

In [ ]:
# ============================================================
# MAPPING: FIELDS WE NEED FOR OUR RATIOS
# ============================================================

# Let's check if the fields we need actually exist in the data
# This helps us catch any naming issues before we write more code

# Fields we expect to need:
needed_fields = {
    'balance_sheet': [
        'Current Assets',
        'Current Liabilities',
        'Total Debt',
        'Stockholders Equity',
        'Cash Cash Equivalents And Short Term Investments'
    ],
    'income_statement': [
        'Total Revenue',
        'Operating Income',
        'Interest Expense'
    ],
    'cash_flow': [
        'Operating Cash Flow',
        'Capital Expenditure'
    ]
}

# Check each one
print("Field Availability Check:")
print("=" * 50)

print("\nBALANCE SHEET:")
for field in needed_fields['balance_sheet']:
    status = "FOUND" if field in balance_sheet.index else "NOT FOUND"
    print(f"  {field}: {status}")

print("\nINCOME STATEMENT:")
for field in needed_fields['income_statement']:
    status = "FOUND" if field in income_statement.index else "NOT FOUND"
    print(f"  {field}: {status}")

print("\nCASH FLOW:")
for field in needed_fields['cash_flow']:
    status = "FOUND" if field in cash_flow.index else "NOT FOUND"
    print(f"  {field}: {status}")

---

## Part 3: Calculating Financial Metrics

Now that we know the field names, we can calculate our ratios. We'll build this up piece by piece:

1. First, calculate ratios for one company (Apple) to make sure the math works
2. Then, expand to all six companies

We'll organize the metrics into three categories:
- **Liquidity** — Can they pay short-term bills?
- **Leverage** — How much debt are they carrying?
- **Cash Flow** — Do they generate real cash?

### 3.1 Liquidity Ratios

Liquidity ratios tell us if a company can pay its short-term obligations. In a downturn, this is critical—companies with poor liquidity can go bankrupt even if they're profitable on paper.

**Current Ratio** = Current Assets / Current Liabilities
- Measures ability to pay bills due within one year
- Above 1.0 means they can cover obligations
- Above 1.5 is generally considered healthy

**Quick Ratio** = (Current Assets - Inventory) / Current Liabilities
- More conservative than Current Ratio
- Excludes inventory because it's harder to convert to cash quickly
- Above 1.0 is considered healthy

In [ ]:
# ============================================================
# CALCULATE: LIQUIDITY RATIOS FOR APPLE
# ============================================================

# We'll use the most recent year's data
# In yfinance, the first column (index 0) is the most recent
most_recent_column = 0

# Pull the values we need from the balance sheet
# .iloc[row, column] lets us grab a specific cell
# We use .loc[field_name] to get the row, then .iloc[0] for first column
current_assets = balance_sheet.loc['Current Assets'].iloc[most_recent_column]
current_liabilities = balance_sheet.loc['Current Liabilities'].iloc[most_recent_column]

# For Quick Ratio, we need inventory
# Some tech companies have zero inventory, so we need to handle that
if 'Inventory' in balance_sheet.index:
    inventory = balance_sheet.loc['Inventory'].iloc[most_recent_column]
else:
    inventory = 0  # If no inventory field, assume zero

# Calculate the ratios
current_ratio = current_assets / current_liabilities
quick_ratio = (current_assets - inventory) / current_liabilities

# Display the results
print("Apple (AAPL) - Liquidity Ratios")
print("=" * 40)
print(f"\nCurrent Assets:      ${current_assets / 1e9:.1f} billion")
print(f"Current Liabilities: ${current_liabilities / 1e9:.1f} billion")
print(f"Inventory:           ${inventory / 1e9:.1f} billion")
print(f"\nCurrent Ratio: {current_ratio:.2f}")
print(f"Quick Ratio:   {quick_ratio:.2f}")

# Add interpretation
print("\n" + "-" * 40)
print("Interpretation:")
if current_ratio >= 1.5:
    print(f"  Current Ratio of {current_ratio:.2f} is healthy (above 1.5)")
elif current_ratio >= 1.0:
    print(f"  Current Ratio of {current_ratio:.2f} is adequate (above 1.0)")
else:
    print(f"  Current Ratio of {current_ratio:.2f} is concerning (below 1.0)")

### 3.2 Leverage Ratios

Leverage ratios tell us how much a company relies on borrowed money. High debt isn't always bad—it can amplify returns—but in a downturn, debt payments don't go away even when revenue drops.

**Debt-to-Equity Ratio** = Total Debt / Stockholders Equity
- Compares what they owe to what shareholders own
- Below 0.5 is conservative
- Below 1.0 is generally acceptable
- Above 2.0 is highly leveraged

**Interest Coverage Ratio** = Operating Income / Interest Expense
- Measures ability to pay interest on debt
- Above 5x is comfortable
- Below 2x is a warning sign

In [ ]:
# ============================================================
# CALCULATE: LEVERAGE RATIOS FOR APPLE
# ============================================================

# Pull values from balance sheet
total_debt = balance_sheet.loc['Total Debt'].iloc[most_recent_column]
stockholders_equity = balance_sheet.loc['Stockholders Equity'].iloc[most_recent_column]

# Pull values from income statement
operating_income = income_statement.loc['Operating Income'].iloc[most_recent_column]
interest_expense = income_statement.loc['Interest Expense'].iloc[most_recent_column]

# Interest expense is sometimes reported as negative (an expense)
# We need the absolute value for our calculation
interest_expense = abs(interest_expense)

# Calculate the ratios
debt_to_equity = total_debt / stockholders_equity

# For interest coverage, we need to handle the case where interest expense is zero
if interest_expense > 0:
    interest_coverage = operating_income / interest_expense
else:
    interest_coverage = float('inf')  # No debt = infinite coverage

# Display the results
print("Apple (AAPL) - Leverage Ratios")
print("=" * 40)
print(f"\nTotal Debt:          ${total_debt / 1e9:.1f} billion")
print(f"Stockholders Equity: ${stockholders_equity / 1e9:.1f} billion")
print(f"Operating Income:    ${operating_income / 1e9:.1f} billion")
print(f"Interest Expense:    ${interest_expense / 1e9:.1f} billion")
print(f"\nDebt-to-Equity Ratio: {debt_to_equity:.2f}")
if interest_coverage == float('inf'):
    print(f"Interest Coverage:    N/A (no interest expense)")
else:
    print(f"Interest Coverage:    {interest_coverage:.1f}x")

# Add interpretation
print("\n" + "-" * 40)
print("Interpretation:")
if debt_to_equity <= 0.5:
    print(f"  Debt-to-Equity of {debt_to_equity:.2f} is conservative")
elif debt_to_equity <= 1.0:
    print(f"  Debt-to-Equity of {debt_to_equity:.2f} is moderate")
else:
    print(f"  Debt-to-Equity of {debt_to_equity:.2f} is elevated")

### 3.3 Cash Flow Metrics

Cash flow metrics tell us about actual cash generation, not just accounting profits. A company can be "profitable" on paper but still run out of cash. In a downturn, cash is survival.

**Free Cash Flow (FCF)** = Operating Cash Flow - Capital Expenditure
- Cash left after running the business and maintaining equipment
- This is cash available for dividends, buybacks, debt repayment, or savings

**FCF Margin** = Free Cash Flow / Total Revenue
- What percentage of revenue converts to free cash
- Higher is better
- Above 20% is excellent for tech companies

**Cash on Hand**
- Cash, cash equivalents, and short-term investments on the balance sheet
- Includes marketable securities that can be quickly converted to cash
- The ultimate safety buffer

In [ ]:
# ============================================================
# CALCULATE: CASH FLOW METRICS FOR APPLE
# ============================================================

# Pull values from cash flow statement
operating_cash_flow = cash_flow.loc['Operating Cash Flow'].iloc[most_recent_column]
capital_expenditure = cash_flow.loc['Capital Expenditure'].iloc[most_recent_column]

# Capital expenditure is typically reported as negative (cash going out)
# Free Cash Flow = Operating Cash Flow - CapEx
# Since CapEx is negative, we add it: OCF + (-CapEx) = OCF - CapEx
free_cash_flow = operating_cash_flow + capital_expenditure

# Pull revenue for FCF Margin calculation
total_revenue = income_statement.loc['Total Revenue'].iloc[most_recent_column]

# Calculate FCF Margin
fcf_margin = free_cash_flow / total_revenue

# Pull cash on hand from balance sheet
# Using the more complete field that includes short-term investments
cash_on_hand = balance_sheet.loc['Cash Cash Equivalents And Short Term Investments'].iloc[most_recent_column]

# Display the results
print("Apple (AAPL) - Cash Flow Metrics")
print("=" * 40)
print(f"\nOperating Cash Flow: ${operating_cash_flow / 1e9:.1f} billion")
print(f"Capital Expenditure: ${capital_expenditure / 1e9:.1f} billion")
print(f"Free Cash Flow:      ${free_cash_flow / 1e9:.1f} billion")
print(f"\nTotal Revenue:       ${total_revenue / 1e9:.1f} billion")
print(f"FCF Margin:          {fcf_margin * 100:.1f}%")
print(f"\nCash on Hand:        ${cash_on_hand / 1e9:.1f} billion")

# Add interpretation
print("\n" + "-" * 40)
print("Interpretation:")
if fcf_margin >= 0.20:
    print(f"  FCF Margin of {fcf_margin*100:.1f}% is excellent")
elif fcf_margin >= 0.10:
    print(f"  FCF Margin of {fcf_margin*100:.1f}% is solid")
elif fcf_margin >= 0:
    print(f"  FCF Margin of {fcf_margin*100:.1f}% is weak")
else:
    print(f"  FCF Margin of {fcf_margin*100:.1f}% is negative (burning cash)")

---

## Part 4: Analyzing All Six Companies

Now that we've confirmed our calculations work for Apple, let's expand to all six companies. We'll:

1. Create a function that calculates all metrics for any ticker
2. Loop through all six companies
3. Build a comparison table

This is where we turn individual calculations into a reusable analysis tool.

### 4.1 Define Our Companies

First, let's set up a list of the companies we want to analyze.

In [ ]:
# ============================================================
# DEFINE THE COMPANIES WE WANT TO ANALYZE
# ============================================================

# Dictionary mapping ticker symbols to company names
# This makes our output more readable
companies = {
    'AAPL': 'Apple',
    'MSFT': 'Microsoft',
    'GOOGL': 'Alphabet',
    'META': 'Meta',
    'NVDA': 'Nvidia',
    'INTC': 'Intel'
}

print("Companies to analyze:")
print("=" * 30)
for ticker, name in companies.items():
    print(f"  {ticker}: {name}")

### 4.2 Create a Function to Calculate All Metrics

Instead of repeating our code six times, we'll wrap it in a function. This function takes a ticker symbol and returns all the metrics we care about.

We'll also add error handling for missing data (like the NaN interest expense we saw with Apple).

In [ ]:
# ============================================================
# FUNCTION: CALCULATE ALL METRICS FOR A GIVEN TICKER
# ============================================================

def get_financial_metrics(ticker_symbol):
    """
    Calculate financial health metrics for a given company.
    
    Parameters:
        ticker_symbol (str): Stock ticker like 'AAPL' or 'MSFT'
    
    Returns:
        dict: Dictionary containing all calculated metrics
    """
    
    # Create a ticker object to access the company's data
    ticker = yf.Ticker(ticker_symbol)
    
    # Pull the three financial statements
    income_stmt = ticker.financials
    balance = ticker.balance_sheet
    cashflow = ticker.cashflow
    
    # We'll use the most recent year (first column)
    col = 0
    
    # ----------------------------------------------------------
    # HELPER FUNCTION: Safely get a value from a dataframe
    # Returns None if the field doesn't exist or is NaN
    # ----------------------------------------------------------
    def safe_get(df, field_name, column=0):
        try:
            value = df.loc[field_name].iloc[column]
            # Check if the value is NaN
            if pd.isna(value):
                return None
            return value
        except:
            return None
    
    # ----------------------------------------------------------
    # PULL RAW VALUES
    # ----------------------------------------------------------
    
    # Balance sheet items
    current_assets = safe_get(balance, 'Current Assets', col)
    current_liabilities = safe_get(balance, 'Current Liabilities', col)
    inventory = safe_get(balance, 'Inventory', col)
    total_debt = safe_get(balance, 'Total Debt', col)
    stockholders_equity = safe_get(balance, 'Stockholders Equity', col)
    # Using the more complete cash field that includes short-term investments
    cash = safe_get(balance, 'Cash Cash Equivalents And Short Term Investments', col)
    
    # Income statement items
    total_revenue = safe_get(income_stmt, 'Total Revenue', col)
    operating_income = safe_get(income_stmt, 'Operating Income', col)
    interest_expense = safe_get(income_stmt, 'Interest Expense', col)
    
    # Cash flow items
    operating_cash_flow = safe_get(cashflow, 'Operating Cash Flow', col)
    capital_expenditure = safe_get(cashflow, 'Capital Expenditure', col)
    
    # ----------------------------------------------------------
    # CALCULATE METRICS
    # ----------------------------------------------------------
    
    # Initialize results dictionary
    metrics = {
        'ticker': ticker_symbol,
        'current_ratio': None,
        'quick_ratio': None,
        'debt_to_equity': None,
        'interest_coverage': None,
        'free_cash_flow': None,
        'fcf_margin': None,
        'cash_on_hand': None
    }
    
    # Liquidity Ratios
    if current_assets and current_liabilities:
        metrics['current_ratio'] = current_assets / current_liabilities
        
        # For quick ratio, treat missing inventory as zero
        inv = inventory if inventory else 0
        metrics['quick_ratio'] = (current_assets - inv) / current_liabilities
    
    # Leverage Ratios
    if total_debt and stockholders_equity:
        metrics['debt_to_equity'] = total_debt / stockholders_equity
    
    if operating_income and interest_expense:
        # Take absolute value of interest expense (sometimes reported negative)
        metrics['interest_coverage'] = operating_income / abs(interest_expense)
    
    # Cash Flow Metrics
    if operating_cash_flow and capital_expenditure:
        # CapEx is typically negative, so we add it
        fcf = operating_cash_flow + capital_expenditure
        metrics['free_cash_flow'] = fcf
        
        if total_revenue:
            metrics['fcf_margin'] = fcf / total_revenue
    
    if cash:
        metrics['cash_on_hand'] = cash
    
    return metrics

print("Function defined successfully.")

### 4.3 Collect Data for All Companies

Now we loop through each company, calculate the metrics, and store them in a list. This might take a few seconds since we're making API calls for each company.

In [ ]:
# ============================================================
# COLLECT METRICS FOR ALL SIX COMPANIES
# ============================================================

# List to store results
all_metrics = []

print("Fetching data for each company...")
print("=" * 40)

# Loop through each company
for ticker, name in companies.items():
    print(f"  Processing {name} ({ticker})...")
    
    # Calculate metrics for this company
    metrics = get_financial_metrics(ticker)
    
    # Add the company name to the metrics
    metrics['company'] = name
    
    # Add to our list
    all_metrics.append(metrics)

print("\nDone! Data collected for all companies.")

### 4.4 Build the Comparison Table

Let's organize all the metrics into a clean table so we can compare the companies side by side.

In [ ]:
# ============================================================
# CREATE A COMPARISON DATAFRAME
# ============================================================

# Convert our list of dictionaries into a pandas DataFrame
df = pd.DataFrame(all_metrics)

# Set the company name as the index for easier reading
df = df.set_index('company')

# Reorder columns to group related metrics
column_order = [
    'ticker',
    'current_ratio',
    'quick_ratio',
    'debt_to_equity',
    'interest_coverage',
    'free_cash_flow',
    'fcf_margin',
    'cash_on_hand'
]
df = df[column_order]

# Display the raw data first
print("Raw Metrics Comparison")
print("=" * 80)
print(df.to_string())

In [ ]:
# ============================================================
# CREATE A FORMATTED VERSION FOR EASIER READING
# ============================================================

# Make a copy so we don't modify the original
df_display = df.copy()

# Format each column appropriately
# Ratios: 2 decimal places
df_display['current_ratio'] = df_display['current_ratio'].apply(
    lambda x: f"{x:.2f}" if pd.notna(x) else "N/A"
)
df_display['quick_ratio'] = df_display['quick_ratio'].apply(
    lambda x: f"{x:.2f}" if pd.notna(x) else "N/A"
)
df_display['debt_to_equity'] = df_display['debt_to_equity'].apply(
    lambda x: f"{x:.2f}" if pd.notna(x) else "N/A"
)

# Interest coverage: 1 decimal with 'x'
df_display['interest_coverage'] = df_display['interest_coverage'].apply(
    lambda x: f"{x:.1f}x" if pd.notna(x) else "N/A"
)

# Cash values: billions with $ sign
df_display['free_cash_flow'] = df_display['free_cash_flow'].apply(
    lambda x: f"${x/1e9:.1f}B" if pd.notna(x) else "N/A"
)
df_display['cash_on_hand'] = df_display['cash_on_hand'].apply(
    lambda x: f"${x/1e9:.1f}B" if pd.notna(x) else "N/A"
)

# FCF Margin: percentage
df_display['fcf_margin'] = df_display['fcf_margin'].apply(
    lambda x: f"{x*100:.1f}%" if pd.notna(x) else "N/A"
)

# Rename columns for display
df_display.columns = [
    'Ticker',
    'Current Ratio',
    'Quick Ratio',
    'Debt/Equity',
    'Interest Coverage',
    'Free Cash Flow',
    'FCF Margin',
    'Cash on Hand'
]

print("\nFormatted Comparison Table")
print("=" * 90)
print(df_display.to_string())

---

## Part 5: Visualizations

Numbers in a table are useful, but charts tell the story faster. We'll create visualizations that highlight:

1. **Liquidity comparison** — Who can pay their bills?
2. **Leverage comparison** — Who's carrying the most debt risk?
3. **Cash flow comparison** — Who generates the most cash?
4. **Overall ranking** — Who's best positioned for a downturn?

### 5.1 Liquidity Comparison: Current Ratio

The current ratio shows if a company can cover short-term obligations. Above 1.0 means they can; above 1.5 is comfortable.

In [ ]:
# Create images folder to save our charts
import os
os.makedirs('../images', exist_ok=True)
print('Images folder ready.')

In [ ]:
# ============================================================
# CHART 1: CURRENT RATIO COMPARISON
# ============================================================

# Create a figure and axis
# figsize sets the width and height in inches
fig, ax = plt.subplots(figsize=(10, 6))

# Get the data for the chart
# We'll use the original df (not the formatted one) for plotting
company_names = df.index.tolist()
current_ratios = df['current_ratio'].values

# Create a bar chart
# The bars will be colored based on whether they meet the threshold
colors = ['green' if x >= 1.5 else 'orange' if x >= 1.0 else 'red' for x in current_ratios]
bars = ax.bar(company_names, current_ratios, color=colors, edgecolor='black', linewidth=1)

# Add a horizontal line at 1.0 (minimum acceptable)
ax.axhline(y=1.0, color='red', linestyle='--', linewidth=1.5, label='Minimum (1.0)')

# Add a horizontal line at 1.5 (healthy threshold)
ax.axhline(y=1.5, color='green', linestyle='--', linewidth=1.5, label='Healthy (1.5)')

# Add value labels on top of each bar
for bar, value in zip(bars, current_ratios):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
            f'{value:.2f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

# Set labels and title
ax.set_ylabel('Current Ratio', fontsize=12)
ax.set_title('Liquidity Comparison: Current Ratio\nCan They Pay Short-Term Bills?', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')

# Clean up the chart
ax.set_ylim(0, max(current_ratios) * 1.15)  # Add some headroom
plt.tight_layout()
plt.savefig('../images/01_current_ratio.png', dpi=150, bbox_inches='tight')
plt.show()

### 5.2 Leverage Comparison: Debt-to-Equity

Debt-to-equity shows how much the company relies on borrowed money. Lower is generally safer in a downturn.

In [ ]:
# ============================================================
# CHART 2: DEBT-TO-EQUITY COMPARISON
# ============================================================

fig, ax = plt.subplots(figsize=(10, 6))

# Get the data
debt_to_equity = df['debt_to_equity'].values

# Color based on risk level
# Lower debt is better, so we reverse the color logic
colors = ['green' if x <= 0.5 else 'orange' if x <= 1.0 else 'red' for x in debt_to_equity]
bars = ax.bar(company_names, debt_to_equity, color=colors, edgecolor='black', linewidth=1)

# Add threshold lines
ax.axhline(y=0.5, color='green', linestyle='--', linewidth=1.5, label='Conservative (0.5)')
ax.axhline(y=1.0, color='orange', linestyle='--', linewidth=1.5, label='Moderate (1.0)')

# Add value labels
for bar, value in zip(bars, debt_to_equity):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
            f'{value:.2f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

# Labels and title
ax.set_ylabel('Debt-to-Equity Ratio', fontsize=12)
ax.set_title('Leverage Comparison: Debt-to-Equity\nLower = Less Risk in a Downturn', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')

ax.set_ylim(0, max(debt_to_equity) * 1.15)
plt.tight_layout()
plt.savefig('../images/02_debt_to_equity.png', dpi=150, bbox_inches='tight')
plt.show()

### 5.3 Cash Flow Comparison: FCF Margin

FCF margin shows what percentage of revenue converts to free cash. This is the money available for survival, growth, or returning to shareholders.

In [ ]:
# ============================================================
# CHART 3: FCF MARGIN COMPARISON
# ============================================================

fig, ax = plt.subplots(figsize=(10, 6))

# Get the data and convert to percentage
fcf_margins = df['fcf_margin'].values * 100  # Convert to percentage

# Color based on performance
colors = ['green' if x >= 20 else 'orange' if x >= 10 else 'gray' if x >= 0 else 'red' for x in fcf_margins]
bars = ax.bar(company_names, fcf_margins, color=colors, edgecolor='black', linewidth=1)

# Add threshold lines
ax.axhline(y=20, color='green', linestyle='--', linewidth=1.5, label='Excellent (20%)')
ax.axhline(y=0, color='red', linestyle='-', linewidth=2, label='Break-even (0%)')

# Add value labels
for bar, value in zip(bars, fcf_margins):
    # Position label above or below bar depending on if it's positive or negative
    if value >= 0:
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                f'{value:.1f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')
    else:
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() - 2,
                f'{value:.1f}%', ha='center', va='top', fontsize=11, fontweight='bold')

# Labels and title
ax.set_ylabel('FCF Margin (%)', fontsize=12)
ax.set_title('Cash Generation: Free Cash Flow Margin\nHigher = More Cash Available for Survival', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')

plt.tight_layout()
plt.savefig('../images/03_fcf_margin.png', dpi=150, bbox_inches='tight')
plt.show()

### 5.4 Cash Reserves: Raw Cash on Hand

Sometimes the simplest metric matters most: how much cash do they have in the bank?

In [ ]:
# ============================================================
# CHART 4: CASH ON HAND
# ============================================================

fig, ax = plt.subplots(figsize=(10, 6))

# Get the data and convert to billions
cash_billions = df['cash_on_hand'].values / 1e9

# Sort by cash amount for this chart
sorted_indices = cash_billions.argsort()[::-1]  # Descending order
sorted_companies = [company_names[i] for i in sorted_indices]
sorted_cash = cash_billions[sorted_indices]

# Create horizontal bar chart (easier to read company names)
bars = ax.barh(sorted_companies, sorted_cash, color='steelblue', edgecolor='black', linewidth=1)

# Add value labels
for bar, value in zip(bars, sorted_cash):
    ax.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2,
            f'${value:.1f}B', ha='left', va='center', fontsize=11, fontweight='bold')

# Labels and title
ax.set_xlabel('Cash on Hand (Billions USD)', fontsize=12)
ax.set_title('Cash Reserves: Money in the Bank\nThe Ultimate Safety Buffer', fontsize=14, fontweight='bold')

# Add some padding on the right for labels
ax.set_xlim(0, max(sorted_cash) * 1.2)

plt.tight_layout()
plt.savefig('../images/04_cash_on_hand.png', dpi=150, bbox_inches='tight')
plt.show()

---

## Part 6: Conclusions

Now let's answer our original question: **Which tech giant is best positioned for a downturn?**

We'll rank the companies across our three categories and identify the overall winner.

### 6.1 Summary Scorecard

Let's create a simple ranking system to compare the companies objectively.

In [ ]:
# ============================================================
# CREATE A RANKING SCORECARD
# ============================================================

# We'll rank each company 1-6 in each category
# Lower rank = better (1 is best)

# Create a copy of our dataframe for ranking
rankings = pd.DataFrame(index=df.index)

# Liquidity: Higher current ratio is better
rankings['Liquidity Rank'] = df['current_ratio'].rank(ascending=False).astype(int)

# Leverage: Lower debt-to-equity is better
rankings['Leverage Rank'] = df['debt_to_equity'].rank(ascending=True).astype(int)

# Cash Flow: Higher FCF margin is better
rankings['Cash Flow Rank'] = df['fcf_margin'].rank(ascending=False).astype(int)

# Cash Buffer: Higher cash on hand is better
rankings['Cash Buffer Rank'] = df['cash_on_hand'].rank(ascending=False).astype(int)

# Calculate overall score (sum of ranks - lower is better)
rankings['Total Score'] = rankings.sum(axis=1)

# Overall rank based on total score
rankings['Overall Rank'] = rankings['Total Score'].rank(ascending=True).astype(int)

# Sort by overall rank
rankings = rankings.sort_values('Overall Rank')

print("Downturn Readiness Scorecard")
print("=" * 70)
print("(Rank 1 = Best in category, 6 = Worst)")
print()
print(rankings.to_string())

In [ ]:
# ============================================================
# VISUALIZE THE OVERALL RANKING
# ============================================================

fig, ax = plt.subplots(figsize=(10, 6))

# Get sorted data
sorted_companies = rankings.index.tolist()
total_scores = rankings['Total Score'].values

# Color by rank (green for best, red for worst)
colors = ['#2ecc71', '#82e0aa', '#f9e79f', '#f5b041', '#e74c3c', '#c0392b']

# Create horizontal bar chart
bars = ax.barh(sorted_companies[::-1], total_scores[::-1], color=colors[::-1], edgecolor='black', linewidth=1)

# Add rank labels
for i, (bar, company) in enumerate(zip(bars, sorted_companies[::-1])):
    rank = len(sorted_companies) - i
    ax.text(bar.get_width() + 0.3, bar.get_y() + bar.get_height()/2,
            f'#{rank}', ha='left', va='center', fontsize=12, fontweight='bold')

# Labels and title
ax.set_xlabel('Total Score (Lower = Better Positioned)', fontsize=12)
ax.set_title('Overall Downturn Readiness Ranking\nBased on Liquidity, Leverage, and Cash Flow', fontsize=14, fontweight='bold')

# Add some padding
ax.set_xlim(0, max(total_scores) * 1.15)

plt.tight_layout()
plt.show()

### 6.2 Key Findings

In [ ]:
# ============================================================
# PRINT KEY FINDINGS
# ============================================================

# Get the top and bottom performers
best_company = rankings.index[0]
worst_company = rankings.index[-1]

print("=" * 60)
print("KEY FINDINGS")
print("=" * 60)

print(f"\n🏆 BEST POSITIONED: {best_company}")
print(f"   - Current Ratio: {df.loc[best_company, 'current_ratio']:.2f}")
print(f"   - Debt-to-Equity: {df.loc[best_company, 'debt_to_equity']:.2f}")
print(f"   - FCF Margin: {df.loc[best_company, 'fcf_margin']*100:.1f}%")
print(f"   - Cash on Hand: ${df.loc[best_company, 'cash_on_hand']/1e9:.1f}B")

print(f"\n⚠️  MOST VULNERABLE: {worst_company}")
print(f"   - Current Ratio: {df.loc[worst_company, 'current_ratio']:.2f}")
print(f"   - Debt-to-Equity: {df.loc[worst_company, 'debt_to_equity']:.2f}")
print(f"   - FCF Margin: {df.loc[worst_company, 'fcf_margin']*100:.1f}%")
print(f"   - Cash on Hand: ${df.loc[worst_company, 'cash_on_hand']/1e9:.1f}B")

print("\n" + "-" * 60)
print("NOTABLE OBSERVATIONS:")
print("-" * 60)
print(f"\n1. NVIDIA leads in liquidity ({df.loc['Nvidia', 'current_ratio']:.2f}) and FCF margin ({df.loc['Nvidia', 'fcf_margin']*100:.1f}%),")
print("   making it the strongest cash generator relative to revenue.")
print(f"\n2. ALPHABET has the lowest debt-to-equity ({df.loc['Alphabet', 'debt_to_equity']:.2f}), meaning")
print("   almost no reliance on borrowed money.")
print(f"\n3. ALPHABET has the most cash on hand (${df.loc['Alphabet', 'cash_on_hand']/1e9:.1f}B),")
print(f"   followed closely by Microsoft (${df.loc['Microsoft', 'cash_on_hand']/1e9:.1f}B).")
print(f"\n4. APPLE shows a weak current ratio ({df.loc['Apple', 'current_ratio']:.2f}) but this is")
print("   intentional—they run lean because cash flow is predictable.")
print(f"\n5. INTEL is in a precarious position with negative FCF ({df.loc['Intel', 'fcf_margin']*100:.1f}%)")
print(f"   and the lowest cash reserves (${df.loc['Intel', 'cash_on_hand']/1e9:.1f}B).")

### 6.3 Final Answer

**Which tech giant is best positioned for a downturn?**

In [ ]:
# ============================================================
# FINAL ANSWER
# ============================================================

print("=" * 60)
print("FINAL ANSWER")
print("=" * 60)
print(f"\nBased on our analysis of liquidity, leverage, and cash flow,")
print(f"{best_company.upper()} is the best positioned for a downturn.")
print("\nHowever, the answer depends on what you prioritize:")
print(f"\n  • Best LIQUIDITY:    Nvidia (Current Ratio: {df.loc['Nvidia', 'current_ratio']:.2f})")
print(f"  • Lowest DEBT:       Alphabet (Debt/Equity: {df.loc['Alphabet', 'debt_to_equity']:.2f})")
print(f"  • Best CASH FLOW:    Nvidia (FCF Margin: {df.loc['Nvidia', 'fcf_margin']*100:.1f}%)")
print(f"  • Most CASH BUFFER:  Alphabet (${df.loc['Alphabet', 'cash_on_hand']/1e9:.1f}B on hand)")
print("\n" + "-" * 60)
print("\nThe WORST positioned is clearly Intel, which is burning cash")
print("while attempting an expensive manufacturing turnaround.")
print("A recession would make their situation significantly harder.")

---

## Limitations & Caveats

This analysis has several limitations worth noting:

1. **Point-in-time snapshot**: Financial health changes over time. This analysis uses the most recent annual data available.

2. **Ratios don't tell the whole story**: Apple's low current ratio looks bad on paper, but their predictable cash flow makes it a non-issue.

3. **Industry context matters**: Comparing hardware companies (Apple, Nvidia, Intel) to software/advertising companies (Microsoft, Alphabet, Meta) has inherent limitations.

4. **Data source**: Yahoo Finance data via yfinance may have inconsistencies. For professional analysis, verify against SEC filings.

5. **Excluded factors**: This analysis doesn't consider revenue diversification, competitive position, management quality, or macroeconomic exposure.

---

## Data Source

All financial data was retrieved from Yahoo Finance using the `yfinance` Python library. Data reflects the most recent annual financial statements available at the time of analysis.